In [2]:
import keras
import torch
import transformers
import pandas as pd
from datasets import Dataset
import torch
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding
from datasets import load_from_disk


2023-12-05 08:20:27.075965: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-05 08:20:27.517903: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
dataset = load_from_disk('data/distilbert-base-uncased_tokenized_dataset')
dataset

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 7887
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 1972
    })
})

In [4]:
dataset.column_names

{'train': ['labels', 'input_ids', 'attention_mask'],
 'test': ['labels', 'input_ids', 'attention_mask']}

In [5]:
#create data collator
model_name = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

#create data loader
train_dataloader = DataLoader(dataset['train'], batch_size=16, collate_fn=data_collator)

In [6]:
for batch in train_dataloader:
    print(batch)
    break

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'labels': tensor([[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
         0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
         1, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
         0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 

In [7]:
labels = batch.pop('labels')

In [8]:
batch

{'input_ids': tensor([[  101,  1041,  3676,  ...,     0,     0,     0],
        [  101,  6622,  1997,  ...,     0,     0,     0],
        [  101,  3795,  1060,  ...,     0,     0,     0],
        ...,
        [  101,  2019,  2552,  ...,     0,     0,     0],
        [  101,  3010, 12012,  ...,     0,     0,     0],
        [  101,  4861,  2012,  ...,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [9]:
#load model
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=50, problem_type='multi_label_classification')

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'classifier.weight', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
output = model(**batch)

In [11]:
with torch.no_grad():
    output = model(**batch)
print(output)

SequenceClassifierOutput(loss=None, logits=tensor([[-5.3810e-02, -6.5098e-02,  4.5533e-02, -2.7215e-02,  2.6312e-02,
          4.3021e-03,  1.1469e-01, -7.3923e-05,  1.0586e-01, -9.3026e-02,
         -3.2912e-02,  4.3366e-02, -3.0034e-02,  1.4252e-01, -1.4582e-01,
          1.7466e-01,  6.4193e-02,  1.5631e-01,  9.6937e-02,  6.6014e-02,
          3.7520e-02,  4.4540e-02,  1.2697e-01, -4.9129e-02, -1.9707e-01,
          5.7044e-02, -3.9536e-02,  6.9872e-02,  5.9503e-02,  6.7755e-02,
          5.4143e-02,  7.1272e-02,  8.7636e-02, -1.1233e-01, -1.5267e-01,
         -3.9699e-02,  9.4313e-02,  7.0356e-02,  5.7860e-02, -8.4137e-02,
          7.9250e-02, -4.7304e-02, -9.6850e-02,  3.7648e-02, -1.0777e-02,
         -3.1993e-02,  5.4081e-02, -6.4268e-02, -2.4575e-03,  2.0928e-03],
        [-6.7155e-02, -1.0279e-02,  2.6296e-02, -2.3047e-02, -7.4425e-03,
         -2.6223e-03,  9.4427e-02, -1.0079e-02,  6.4856e-02, -4.0869e-02,
          8.6453e-03,  3.1303e-02,  1.2334e-03,  1.2009e-01, -1.4612

In [12]:
labels

tensor([[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
         0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
         1, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
         0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0,

In [13]:
#calculate loss for multi-label classification
import torch.nn as nn
loss = nn.BCEWithLogitsLoss()
loss(output.logits, labels.float())

tensor(0.6998)